In [1]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import FileDatasetFactory

# Connect to your Azure ML Workspace
workspace = Workspace.from_config()

datastore = workspace.get_default_datastore()

# Define paths for data in Azure Blob Storage
datastore_path = "script-mode-workflow/pickle"
train_data_path = os.path.join(datastore_path, "train")
# Assuming your file is in a local directory named "train_dir" that contains "train.pickle"
#local_train_dir = os.path.join(os.getcwd(), "train_dir")
local_train_dir = os.getcwd()

In [10]:
import numpy as np
import pickle
def pickleTrainingData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    train_data = np.empty((0, 32*32*3))
    train_labels = []

    for i in range(1, 2):
        fileNameDataBatch = './cifar-10-batches-py/data_batch_' + str(i)
        batch = unpickle(fileNameDataBatch)
        train_data = np.vstack((train_data, batch[b'data']))
        train_labels += batch[b'labels']

    train_labels = np.array(train_labels)
    train_data = train_data.reshape(-1, 32, 32, 3) / 255.0
    
    # !!!!! NOTICE HOW THE DATA IS SAVED !!!!!!
    # Will be returned in form of:
    # train_label, train_data  = getDataBack()
    pickle.dump([train_labels,train_data], open('./train.cnn', 'wb'))



# CODE HELPER 2
def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])
    
    num_samples_to_select = 600
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]
    
    return selected_test_data, selected_test_labels


In [ ]:
train, label = pickleTrainingData()

In [2]:
#Upload the entire directory to the datastore
FileDatasetFactory.upload_directory(src_dir=local_train_dir,
                                    target=(datastore, train_data_path),
                                    overwrite=True)

# Create a FileDataset pointing to the uploaded directory/files
train_dataset = Dataset.File.from_files(path=(datastore, train_data_path))

# This path can be used to access the dataset during training
datastore_train_path = train_dataset.as_mount()

Validating arguments.
Arguments validated.
'overwrite' is set to True. Any file already present in the target will be overwritten.
Uploading files from '/mnt/batch/tasks/shared/LS_root/mounts/clusters/hmartin112/code/Users/hmartin11' to 'script-mode-workflow/pickle/train'
Copying 25 files with concurrency set to 4
Copied /mnt/batch/tasks/shared/LS_root/mounts/clusters/hmartin112/code/Users/hmartin11/.ipynb_checkpoints/conda_dependency_file-checkpoint.yml, file 1 out of 25. Destination path: https://a4q32845494989.blob.core.windows.net/azureml-blobstore-0024d333-c080-4c76-aca6-074663f996c9/script-mode-workflow/pickle/train/.ipynb_checkpoints/conda_dependency_file-checkpoint.yml
Copied /mnt/batch/tasks/shared/LS_root/mounts/clusters/hmartin112/code/Users/hmartin11/.ipynb_checkpoints/cnnDeployment-checkpoint.ipynb, file 2 out of 25. Destination path: https://a4q32845494989.blob.core.windows.net/azureml-blobstore-0024d333-c080-4c76-aca6-074663f996c9/script-mode-workflow/pickle/train/.ipynb

In [3]:
hyperparameters = {
    "--copy_X": True,
    "--fit_intercept": True,
}

In [4]:
env = Environment.from_conda_specification(name="sklearn-env", file_path="./conda_dependency_file.yml")
arguments = [f'{k}={v}' for k, v in hyperparameters.items()]
arguments.extend(['--train', datastore_train_path])

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
src = ScriptRunConfig(source_directory='./scripts',
                      script='script.py',
                      arguments=arguments,
                      compute_target='local',  # Using the local compute (the Compute Instance)
                      environment=env)


experiment = Experiment(workspace, 'modelCNN')

In [7]:
#conda install -c conda-forge tensorflow

Channels:
 - conda-forge
 - defaults
 - anaconda
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [5]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True)

RunId: modelCNN_1731817699_bbdc07db
Web View: https://ml.azure.com/runs/modelCNN_1731817699_bbdc07db?wsid=/subscriptions/40d42a86-f25f-44cb-be23-4440afba7382/resourcegroups/a4-3/workspaces/a4-q3&tid=9be73e3a-3b63-4ea5-8e18-675e687d2de9

Streaming azureml-logs/70_driver_log.txt

[2024-11-17T04:28:21.032293] Entering context manager injector.
[2024-11-17T04:28:21.562139] context_manager_injector.py Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'Dataset:context_managers.Datasets', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['script.py', '--copy_X=True', '--fit_intercept=True', '--train', 'DatasetConsumptionConfig:input__5179abe5'])
[2024-11-17T04:28:21.933] Initialize DatasetContextManager.
Script type = None
Set Dataset input__5179abe5's target path to /tmp/input__5179abe5_modelCNN_1731817699_bbdc07db_de7309a0-918f-4c46-95e0-4755f8496e3b
[2024-11-17T04:28:21.968] Enter __enter__

{'runId': 'modelCNN_1731817699_bbdc07db',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-11-17T04:28:20.873958Z',
 'endTimeUtc': '2024-11-17T04:29:00.51283Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  '_azureml.ClusterName': 'local',
  'ContentSnapshotId': 'e775c1bd-7116-41a1-a007-463482bbe655'},
 'inputDatasets': [{'dataset': {'id': 'de7309a0-918f-4c46-95e0-4755f8496e3b'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__5179abe5', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--copy_X=True',
   '--fit_intercept=True',
   '--train',
   'DatasetConsumptionConfig:input__5179abe5'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'input__5179abe5': {'dataLocation': {'dataset': {'id': 'de7309a0-918f-4c46-95e0-4755f8496e3b',
 

In [6]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

In [7]:
import argparse
import joblib
from azureml.core import Workspace, Model, Run
ws = Workspace.from_config()
model_name = 'model3'
#ws = Run.get_context().experiment.workspace
#model_obj = Model(workspace, model_name )
model_path = Model.get_model_path(model_name = 'model3', version=1, _workspace=ws)  # Replace with your model name
    # Load the model

In [7]:
model_path = Model.get_model_path('model3')

In [8]:
print(model_path)

azureml-models/model3/1/modelCNN


In [8]:
import tensorflow as tf
from tensorflow import keras
loaded_model = tf.keras.models.load_model(model_path)

2024-11-17 06:13:36.531982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-17 06:13:36.594170: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2024-11-17 06:13:38.351263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
import pickle 
import numpy as np

test_data, test_labels = getTestData()
test_loss, test_acc = loaded_model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

19/19 [==============================] - 0s 4ms/step - loss: 214.2594 - accuracy: 0.1150
Test accuracy: 0.11500000208616257
